# Normal

In [2]:
from models.lag_ml_model import scaling_function,preprocess_data,classifier_models,train_model,grid_optimize_model,random_optimize_model,get_param_grids,get_data

In [3]:
from main import normal_model,grid_model,random_model,backtest_loop,backtest

c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\backtesting\_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [4]:
import pandas as pd
pd.set_option('display.max_columns',None)

In [5]:
df = get_data(ind=True)

In [6]:
df

,Date,Close,High,Low,Open,Volume,sentiment_score,shifted_direction,RSI_14,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,ADX_14,DMP_14,DMN_14,EMA_20,BBL_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0,OBV,STOCHk_14_3_3,STOCHd_14_3_3
33,2015-02-20,32.375000,32.375000,32.012501,32.154999,195793600,0.812261,1.0,78.449322,1.371058,0.114467,1.256591,45.149694,41.323149,9.816305,30.397890,27.349326,33.074674,18.950574,0.877794,3.799902e+09,98.106664,96.643931
34,2015-02-23,33.250000,33.250000,32.415001,32.505001,283896400,0.039249,1.0,82.172193,1.432559,0.140774,1.291785,47.338241,48.012042,8.697291,30.669520,27.459506,33.464994,19.714527,0.964200,4.083799e+09,98.507073,97.380392
35,2015-02-24,33.042500,33.400002,32.792500,33.235001,276912400,0.831066,1.0,78.700126,1.447865,0.124864,1.323001,49.388879,46.180459,8.014210,30.895518,27.673430,33.727819,19.720736,0.886806,3.806886e+09,96.882478,97.832072
36,2015-02-25,32.197498,32.900002,32.037498,32.889999,298846800,0.682991,1.0,66.396541,1.375949,0.042359,1.333590,48.808258,40.511878,16.251885,31.019516,28.293439,33.599061,17.144638,0.735834,3.508040e+09,86.313349,93.900967
37,2015-02-26,32.605000,32.717499,31.652500,32.197498,365150000,0.865478,1.0,68.919985,1.336432,0.002274,1.334159,47.281865,35.534169,18.696788,31.170514,28.576379,33.693871,16.436395,0.787226,3.873190e+09,79.325859,87.507229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2549,2025-02-21,245.550003,248.690002,245.220001,245.949997,53197400,0.009649,1.0,61.165895,2.215105,1.910338,0.304767,22.289078,32.068421,14.679193,238.482268,222.532220,249.664779,11.492050,0.848345,5.924017e+09,89.739206,89.325710
2550,2025-02-24,247.100006,248.860001,244.419998,244.929993,51326400,0.700623,1.0,62.870455,2.596254,1.833189,0.763065,23.116808,30.129422,14.881077,239.303005,224.397091,250.231908,10.886321,0.878772,5.975344e+09,90.696345,90.169481
2551,2025-02-25,247.039993,250.000000,244.910004,248.000000,48013300,0.833927,1.0,62.755600,2.860500,1.677949,1.182552,24.054567,29.592297,13.847395,240.039861,225.069643,251.277356,11.003623,0.838316,5.927330e+09,88.478360,89.637970
2552,2025-02-26,240.360001,244.979996,239.130005,244.330002,44433600,0.478114,1.0,51.481777,2.502056,1.055603,1.446452,23.333550,26.510439,20.015294,240.070351,225.139945,251.417054,11.027898,0.579213,5.882897e+09,78.937497,86.037401


In [7]:
X_train,X_test,y_train,y_test = preprocess_data(df,max_lag=60)

In [8]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((1722, 81), (739, 81), (1722,), (739,))

In [29]:
X_train.columns

Index(['Close', 'High', 'Low', 'Open', 'Volume', 'sentiment_score', 'RSI_14',
       'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9', 'ADX_14', 'DMP_14',
       'DMN_14', 'EMA_20', 'BBL_20_2.0', 'BBU_20_2.0', 'BBB_20_2.0',
       'BBP_20_2.0', 'OBV', 'STOCHk_14_3_3', 'STOCHd_14_3_3', 'Close_lag1',
       'Close_lag2', 'Close_lag3', 'Close_lag4', 'Close_lag5', 'Close_lag6',
       'Close_lag7', 'Close_lag8', 'Close_lag9', 'Close_lag10', 'Close_lag11',
       'Close_lag12', 'Close_lag13', 'Close_lag14', 'Close_lag15',
       'Close_lag16', 'Close_lag17', 'Close_lag18', 'Close_lag19',
       'Close_lag20', 'Close_lag21', 'Close_lag22', 'Close_lag23',
       'Close_lag24', 'Close_lag25', 'Close_lag26', 'Close_lag27',
       'Close_lag28', 'Close_lag29', 'Close_lag30', 'Close_lag31',
       'Close_lag32', 'Close_lag33', 'Close_lag34', 'Close_lag35',
       'Close_lag36', 'Close_lag37', 'Close_lag38', 'Close_lag39',
       'Close_lag40', 'Close_lag41', 'Close_lag42', 'Close_lag43',
       '

In [9]:
X_train_scaled,X_test_scaled = scaling_function(X_train,X_test)

In [10]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((1722, 81), (739, 81), (1722,), (739,))

In [11]:
results,y_pred_dict = normal_model(X_train_scaled,X_test_scaled,y_train,y_test)


Logistic Regression Performance:
Accuracy: 0.8620
Confusion Matrix:
[[266  79]
 [ 23 371]]
              precision    recall  f1-score   support

           0       0.92      0.77      0.84       345
           1       0.82      0.94      0.88       394

    accuracy                           0.86       739
   macro avg       0.87      0.86      0.86       739
weighted avg       0.87      0.86      0.86       739


Random Forest Performance:
Accuracy: 0.8173
Confusion Matrix:
[[217 128]
 [  7 387]]
              precision    recall  f1-score   support

           0       0.97      0.63      0.76       345
           1       0.75      0.98      0.85       394

    accuracy                           0.82       739
   macro avg       0.86      0.81      0.81       739
weighted avg       0.85      0.82      0.81       739


XGBoost Performance:
Accuracy: 0.8877
Confusion Matrix:
[[289  56]
 [ 27 367]]
              precision    recall  f1-score   support

           0       0.91      0.84

c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [12]:
results

,Model,Accuracy,Confusion Matrix
0,Logistic Regression,0.861976,"[[266, 79], [23, 371]]"
1,Random Forest,0.817321,"[[217, 128], [7, 387]]"
2,XGBoost,0.887686,"[[289, 56], [27, 367]]"
3,SVM,0.863329,"[[276, 69], [32, 362]]"
4,Naive Bayes,0.849797,"[[247, 98], [13, 381]]"
5,Gradient Boosting,0.880920,"[[281, 64], [24, 370]]"
6,K-Nearest Neighbors,0.864682,"[[269, 76], [24, 370]]"
7,Decision Tree,0.856563,"[[299, 46], [60, 334]]"
8,AdaBoost,0.878214,"[[293, 52], [38, 356]]"
9,SGD Classifier,0.891746,"[[301, 44], [36, 358]]"


In [13]:
normal_df,bt_collection = backtest_loop(y_pred_dict,X_test)

Backtesting for Logistic Regression


Backtesting for Random Forest


Backtesting for XGBoost


Backtesting for SVM


Backtesting for Naive Bayes


...

Backtesting for Decision Tree


Backtesting for AdaBoost


Backtesting for SGD Classifier


Backtesting for MLP


Backtesting for LGBMClassifier


In [14]:
normal_df

,Start,End,Duration,Exposure Time [%],Equity Final [$],Equity Peak [$],Return [%],Buy & Hold Return [%],Return (Ann.) [%],Volatility (Ann.) [%],CAGR [%],Sharpe Ratio,Sortino Ratio,Calmar Ratio,Alpha [%],Beta,Max. Drawdown [%],Avg. Drawdown [%],Max. Drawdown Duration,Avg. Drawdown Duration,# Trades,Win Rate [%],Best Trade [%],Worst Trade [%],Avg. Trade [%],Max. Trade Duration,Avg. Trade Duration,Profit Factor,Expectancy [%],SQN,Kelly Criterion,_strategy,_equity_curve,_trades,Model
0,2022-03-18,2025-02-27,1077 days,62.516915,13207.580185,14186.071014,32.075802,44.712775,9.951433,19.129675,6.726096,0.520209,0.848220,0.525042,14.028189,0.403634,-18.953608,-4.357536,340 days,62 days,24,37.500000,20.392654,-7.227740,1.058319,146 days,27 days,1.747583,1.265760,0.788924,0.141454,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,Logistic Regression
1,2022-03-18,2025-02-27,1077 days,70.906631,12720.429459,13425.797882,27.204295,44.712775,8.551354,20.851264,5.791718,0.410112,0.642020,0.357357,4.958536,0.497526,-23.929459,-5.879222,299 days,62 days,31,29.032258,31.863605,-7.227740,0.732529,196 days,24 days,1.553509,0.995546,0.588325,0.083841,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,Random Forest
2,2022-03-18,2025-02-27,1077 days,59.539919,11544.361633,13230.371307,15.443616,44.712775,5.019085,18.381369,3.417377,0.273053,0.413214,0.188472,-3.180998,0.416539,-26.630378,-6.357648,322 days,81 days,27,37.037037,23.027569,-7.227740,0.601042,157 days,23 days,1.463618,0.784182,0.538096,0.098384,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,XGBoost
3,2022-03-18,2025-02-27,1077 days,60.216509,11828.309921,13795.960190,18.283099,44.712775,5.892875,18.732674,4.007029,0.314577,0.471631,0.282124,-0.088051,0.410870,-20.887540,-5.289994,337 days,70 days,26,34.615385,20.392654,-7.247327,0.548294,146 days,24 days,1.428167,0.701162,0.503993,0.085778,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,SVM
4,2022-03-18,2025-02-27,1077 days,65.899865,12858.899521,15330.739090,28.588995,44.712775,8.952862,21.627111,6.060059,0.413965,0.668095,0.462555,4.764514,0.532834,-19.355229,-4.794804,226 days,43 days,21,28.571429,36.097860,-11.187451,1.093748,207 days,33 days,1.715683,1.535615,0.550577,0.095325,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,Naive Bayes
5,2022-03-18,2025-02-27,1077 days,60.351827,11275.387207,14695.027756,12.753872,44.712775,4.178212,18.696943,2.848483,0.223470,0.324139,0.179548,-6.063169,0.420843,-23.270732,-5.129384,301 days,58 days,22,50.000000,18.979983,-11.267103,0.628015,154 days,29 days,1.431366,0.870124,0.389990,0.110534,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,Gradient Boosting
6,2022-03-18,2025-02-27,1077 days,61.705007,12629.009506,13708.208969,26.290095,44.712775,8.284692,19.878226,5.613326,0.416772,0.668854,0.383398,6.205627,0.449189,-21.608600,-5.221386,340 days,70 days,22,40.909091,18.427734,-7.108138,0.951205,154 days,29 days,1.590822,1.182039,0.650311,0.130007,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,K-Nearest Neighbors
7,2022-03-18,2025-02-27,1077 days,53.856563,11628.970322,13205.791046,16.289703,44.712775,5.280917,17.776908,3.594228,0.297066,0.467745,0.216819,-1.057814,0.387977,-24.356360,-4.418977,577 days,71 days,28,39.285714,18.427734,-5.848937,0.601427,154 days,20 days,1.480000,0.763638,0.557931,0.106914,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,Decision Tree
8,2022-03-18,2025-02-27,1077 days,56.156969,11515.740311,13894.909790,15.157403,44.712775,4.930226,17.646879,3.357328,0.279382,0.425968,0.237909,-2.333371,0.391181,-20.723145,-4.500793,318 days,58 days,17,35.294118,29.805437,-7.247327,0.939220,195 days,35 days,1.639598,1.264628,0.494252,0.113763,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar Entry

In [25]:
stats,_ = backtest(y_test,X_test)

In [26]:
stats = pd.DataFrame(stats).T
stats['Model'] = 'Actual'

In [27]:
stats

,Start,End,Duration,Exposure Time [%],Equity Final [$],Equity Peak [$],Return [%],Buy & Hold Return [%],Return (Ann.) [%],Volatility (Ann.) [%],CAGR [%],Sharpe Ratio,Sortino Ratio,Calmar Ratio,Alpha [%],Beta,Max. Drawdown [%],Avg. Drawdown [%],Max. Drawdown Duration,Avg. Drawdown Duration,# Trades,Win Rate [%],Best Trade [%],Worst Trade [%],Avg. Trade [%],Max. Trade Duration,Avg. Trade Duration,Profit Factor,Expectancy [%],SQN,Kelly Criterion,_strategy,_equity_curve,_trades,Model
0,2022-03-18 00:00:00,2025-02-27 00:00:00,1077 days 00:00:00,51.962111,17565.242096,18567.572235,75.652421,44.712775,21.179008,20.663597,14.089317,1.024943,1.984313,1.59779,58.034279,0.394029,-13.255185,-2.912822,226 days 00:00:00,31 days 00:00:00,12,50.0,34.317992,-7.722031,4.72902,193 days 00:00:00,45 days 00:00:00,3.64031,5.394187,1.340458,0.333062,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,Actual


In [28]:
pd.concat([normal_df,stats],axis=0)

,Start,End,Duration,Exposure Time [%],Equity Final [$],Equity Peak [$],Return [%],Buy & Hold Return [%],Return (Ann.) [%],Volatility (Ann.) [%],CAGR [%],Sharpe Ratio,Sortino Ratio,Calmar Ratio,Alpha [%],Beta,Max. Drawdown [%],Avg. Drawdown [%],Max. Drawdown Duration,Avg. Drawdown Duration,# Trades,Win Rate [%],Best Trade [%],Worst Trade [%],Avg. Trade [%],Max. Trade Duration,Avg. Trade Duration,Profit Factor,Expectancy [%],SQN,Kelly Criterion,_strategy,_equity_curve,_trades,Model
0,2022-03-18 00:00:00,2025-02-27 00:00:00,1077 days 00:00:00,62.516915,13207.580185,14186.071014,32.075802,44.712775,9.951433,19.129675,6.726096,0.520209,0.84822,0.525042,14.028189,0.403634,-18.953608,-4.357536,340 days 00:00:00,62 days 00:00:00,24,37.5,20.392654,-7.22774,1.058319,146 days 00:00:00,27 days 00:00:00,1.747583,1.26576,0.788924,0.141454,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,Logistic Regression
1,2022-03-18 00:00:00,2025-02-27 00:00:00,1077 days 00:00:00,70.906631,12720.429459,13425.797882,27.204295,44.712775,8.551354,20.851264,5.791718,0.410112,0.64202,0.357357,4.958536,0.497526,-23.929459,-5.879222,299 days 00:00:00,62 days 00:00:00,31,29.032258,31.863605,-7.22774,0.732529,196 days 00:00:00,24 days 00:00:00,1.553509,0.995546,0.588325,0.083841,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,Random Forest
2,2022-03-18 00:00:00,2025-02-27 00:00:00,1077 days 00:00:00,59.539919,11544.361633,13230.371307,15.443616,44.712775,5.019085,18.381369,3.417377,0.273053,0.413214,0.188472,-3.180998,0.416539,-26.630378,-6.357648,322 days 00:00:00,81 days 00:00:00,27,37.037037,23.027569,-7.22774,0.601042,157 days 00:00:00,23 days 00:00:00,1.463618,0.784182,0.538096,0.098384,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,XGBoost
3,2022-03-18 00:00:00,2025-02-27 00:00:00,1077 days 00:00:00,60.216509,11828.309921,13795.96019,18.283099,44.712775,5.892875,18.732674,4.007029,0.314577,0.471631,0.282124,-0.088051,0.41087,-20.88754,-5.289994,337 days 00:00:00,70 days 00:00:00,26,34.615385,20.392654,-7.247327,0.548294,146 days 00:00:00,24 days 00:00:00,1.428167,0.701162,0.503993,0.085778,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,SVM
4,2022-03-18 00:00:00,2025-02-27 00:00:00,1077 days 00:00:00,65.899865,12858.899521,15330.73909,28.588995,44.712775,8.952862,21.627111,6.060059,0.413965,0.668095,0.462555,4.764514,0.532834,-19.355229,-4.794804,226 days 00:00:00,43 days 00:00:00,21,28.571429,36.09786,-11.187451,1.093748,207 days 00:00:00,33 days 00:00:00,1.715683,1.535615,0.550577,0.095325,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,Naive Bayes
5,2022-03-18 00:00:00,2025-02-27 00:00:00,1077 days 00:00:00,60.351827,11275.387207,14695.027756,12.753872,44.712775,4.178212,18.696943,2.848483,0.22347,0.324139,0.179548,-6.063169,0.420843,-23.270732,-5.129384,301 days 00:00:00,58 days 00:00:00,22,50.0,18.979983,-11.267103,0.628015,154 days 00:00:00,29 days 00:00:00,1.431366,0.870124,0.38999,0.110534,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,Gradient Boosting
6,2022-03-18 00:00:00,2025-02-27 00:00:00,1077 days 00:00:00,61.705007,12629.009506,13708.208969,26.290095,44.712775,8.284692,19.878226,5.613326,0.416772,0.668854,0.383398,6.205627,0.449189,-21.6086,-5.221386,340 days 00:00:00,70 days 00:00:00,22,40.909091,18.427734,-7.108138,0.951205,154 days 00:00:00,29 days 00:00:00,1.590822,1.182039,0.650311,0.130007,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,K-Nearest Neighbors
7,2022-03-18 00:00:00,2025-02-27 00:00:00,1077 days 00:00:00,53.856563,11628.970322,13205.791046,16.289703,44.712775,5.280917,17.776908,3.594228,0.297066,0.467745,0.216819,-1.057814,0.387977,-24.35636,-4.418977,577 days 00:00:00,71 days 00:00:00,28,39.285714,18.427734,-5.848937,0.601427,154 days 00:00:00,20 days 00:00:00,1.48,0.763638,0.55793

# Random

In [30]:
rand_results,y_pred_dict_rand = random_model(X_train_scaled=X_train_scaled, X_test_scaled=X_test_scaled,y_train=y_train,y_test=y_test)

Working on optimizing with RandomSearchCV
Optimizing Logistic Regression...
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best parameters for Logistic Regression: {'solver': 'saga', 'penalty': 'l1', 'C': 0.1}
Optimizing Random Forest...
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best parameters for Random Forest: {'n_estimators': 100, 'min_samples_split': 2, 'max_features': 'sqrt', 'max_depth': 10}
Optimizing XGBoost...
No hyperparameters to tune for XGBoost
Optimizing SVM...
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best parameters for SVM: {'kernel': 'linear', 'gamma': 'scale', 'C': 0.1}
Optimizing Naive Bayes...
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best parameters for Naive Bayes: {'alpha': 0.1}
Optimizing Gradient Boosting...
Fitting 5 folds for each of 10 candidates, totalling 50 fits


c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\model_selection\_search.py:317: UserWarning: The total space of parameters 3 is smaller than n_iter=10. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best parameters for Gradient Boosting: {'n_estimators': 100, 'min_samples_split': 2, 'max_depth': 3, 'learning_rate': 0.05}
Optimizing K-Nearest Neighbors...
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best parameters for K-Nearest Neighbors: {'weights': 'distance', 'p': 2, 'n_neighbors': 7, 'algorithm': 'kd_tree'}
Optimizing Decision Tree...
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best parameters for Decision Tree: {'min_samples_split': 2, 'max_features': None, 'max_depth': 10, 'criterion': 'entropy'}
Optimizing AdaBoost...
Fitting 5 folds for each of 6 candidates, totalling 30 fits


c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\model_selection\_search.py:317: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best parameters for AdaBoost: {'n_estimators': 50, 'learning_rate': 0.1}
Optimizing SGD Classifier...
Fitting 5 folds for each of 10 candidates, totalling 50 fits


c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
25 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\linear_model\_stochast

Best parameters for SGD Classifier: {'penalty': 'l2', 'loss': 'log_loss', 'learning_rate': 'optimal', 'alpha': 0.001}
Optimizing MLP...
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best parameters for MLP: {'solver': 'sgd', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (100,), 'alpha': 0.001, 'activation': 'tanh'}
Optimizing LGBMClassifier...
[LightGBM] [Info] Number of positive: 1095, number of negative: 627
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002242 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 20655
[LightGBM] [Info] Number of data points in the train set: 1722, number of used features: 81
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.635889 -> initscore=0.557563
[LightGBM] [Info] Start training from score 0.557563
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: 

c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


# Random